In [52]:
import arcpy
import os
import math
import datetime
import uuid

In [53]:
def find_field_by_pattern(fields, pattern_list):
    """Helper function to find a field based on patterns"""
    for field in fields:
        field_name = field.name.upper()
        if all(pattern in field_name for pattern in pattern_list):
            return field.name
    return None

In [54]:
# Ask user for threshold values
print("====== PCI Difference Threshold Input ======")
lower_threshold = float(input("Enter LOWER threshold value (include points with PCI difference <= this value): "))
higher_threshold = float(input("Enter HIGHER threshold value (include points with PCI difference >= this value): "))
print(f"Will include points with PCI difference <= {lower_threshold} OR >= {higher_threshold}")

# New input for work history filtering
work_history_filter = input("Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): ")
work_history_filter = work_history_filter.lower().strip()
if work_history_filter == 'yes':
    print("Will only include rows above the positive threshold that have NO work history (empty MRTreatmentName)")
else:
    print("Will include all rows that meet threshold criteria regardless of work history")
print("============================================")

====== PCI Difference Threshold Input ======
Enter LOWER threshold value (include points with PCI difference <= this value): -40
Enter HIGHER threshold value (include points with PCI difference >= this value): 40
Will include points with PCI difference <= -40.0 OR >= 40.0
Do you want to only include rows above the positive threshold that have NO work history? (Yes/No): Yes
Will only include rows above the positive threshold that have NO work history (empty MRTreatmentName)


In [55]:
# Get the current project
aprx = arcpy.mp.ArcGISProject("CURRENT")
print(f"Current project: {aprx.filePath}")

# Get the active map and find the WildomarPMPJordan feature layer
PMP_Path = None
active_map = None

# Search for the feature class in all maps
for m in aprx.listMaps():
    for lyr in m.listLayers():
        if lyr.isFeatureLayer and "SSstreet" in lyr.name:
            PMP_Path = lyr.dataSource
            wildomar_layer = lyr
            active_map = m
            print(f"Found layer: {lyr.name} in map: {m.name}")
            break
    if PMP_Path:
        break

if not PMP_Path:
    print("ERROR: WildomarPMPJordan feature class not found in the project.")
    # raise RuntimeError("WildomarPMPJordan feature class not found in the project.")

# Define WGS84 (lat/long) spatial reference for external API URLs
wgs84 = arcpy.SpatialReference(4326)  # EPSG code for WGS84

# Get the spatial reference of the feature layer
desc = arcpy.Describe(wildomar_layer)
layer_sr = desc.spatialReference

print(f"Feature layer spatial reference: {layer_sr.name} ({layer_sr.factoryCode})")
print(f"Target spatial reference: WGS84 (4326)")


Current project: C:\Users\JLin\Downloads\QC Tool\QC Tool.aprx
Found layer: SSstreet in map: Map
Feature layer spatial reference: NAD_1983_StatePlane_California_VI_FIPS_0406_Feet (2230)
Target spatial reference: WGS84 (4326)


In [56]:
# Find the PCI Differences table
pci_table = None
for table in active_map.listTables():
    if "PCI Differences" in table.name:
        pci_table = table
        print(f"Found standalone table: {table.name}")
        break

if not pci_table:
    print("WARNING: PCI Differences table not found in the active map.")
    # Use a raise statement here if you want to stop execution

Found standalone table: PCI Differences


In [57]:
# Determine field names in the feature layer
print("\nLooking for key fields in the feature layer...")
shapefile_fields = arcpy.ListFields(wildomar_layer)

# Try to find the StreetSec field or StreetID/SectionID fields and Street Name
street_sec_field = find_field_by_pattern(shapefile_fields, ["STREETSEC"])
if not street_sec_field:
    street_sec_field = find_field_by_pattern(shapefile_fields, ["STREET", "SEC"]) 

street_id_field = find_field_by_pattern(shapefile_fields, ["STREET", "ID"])
section_id_field = find_field_by_pattern(shapefile_fields, ["SECTION", "ID"])
street_name_field = find_field_by_pattern(shapefile_fields, ["STREET", "NAME"])
begin_loc_field = find_field_by_pattern(shapefile_fields, ["BEGIN", "LOC"])
end_loc_field = find_field_by_pattern(shapefile_fields, ["END", "LOC"])

# Print found fields
if street_sec_field:
    print(f"Found combined street/section field: {street_sec_field}")
if street_id_field:
    print(f"Found street ID field: {street_id_field}")
if section_id_field:
    print(f"Found section ID field: {section_id_field}")
if street_name_field:
    print(f"Found street name field: {street_name_field}")
if begin_loc_field:
    print(f"Found begin location field in feature layer: {begin_loc_field}")
if end_loc_field:
    print(f"Found end location field in feature layer: {end_loc_field}")


Looking for key fields in the feature layer...
Found combined street/section field: StreetSect
Found street ID field: StreetID
Found section ID field: SectionID
Found street name field: STREETNAME
Found end location field in feature layer: EndLocatio


In [58]:
# Determine field names in the PCI table
print("\nLooking for key fields in the PCI Differences table...")
pci_fields = arcpy.ListFields(pci_table)

pci_street_id_field = find_field_by_pattern(pci_fields, ["STREET", "ID"])
pci_section_id_field = find_field_by_pattern(pci_fields, ["SECTION", "ID"])
prev_insp_pci_field = find_field_by_pattern(pci_fields, ["PREV", "PCI"])
last_insp_pci_field = find_field_by_pattern(pci_fields, ["LAST", "PCI"])
prev_insp_date_field = find_field_by_pattern(pci_fields, ["PREV", "DATE"])
m_r_date_field = find_field_by_pattern(pci_fields, ["M&R", "DATE"])
m_r_treatment_field = find_field_by_pattern(pci_fields, ["TREATMENT"])
if not m_r_treatment_field:
    m_r_treatment_field = find_field_by_pattern(pci_fields, ["M&R", "NAME"])
last_insp_date_field = find_field_by_pattern(pci_fields, ["LAST", "DATE"])
begin_loc_field_pci = find_field_by_pattern(pci_fields, ["BEGIN", "LOC"])
end_loc_field_pci = find_field_by_pattern(pci_fields, ["END", "LOC"])

# Print found fields
if pci_street_id_field:
    print(f"Found street ID field: {pci_street_id_field}")
if pci_section_id_field:
    print(f"Found section ID field: {pci_section_id_field}")
if prev_insp_pci_field:
    print(f"Found previous inspection PCI field: {prev_insp_pci_field}")
if last_insp_pci_field:
    print(f"Found last inspection PCI field: {last_insp_pci_field}")
if prev_insp_date_field:
    print(f"Found previous inspection date field: {prev_insp_date_field}")
if m_r_date_field:
    print(f"Found M&R date field: {m_r_date_field}")
if m_r_treatment_field:
    print(f"Found M&R treatment field: {m_r_treatment_field}")
if last_insp_date_field:
    print(f"Found last inspection date field: {last_insp_date_field}")
if begin_loc_field_pci:
    print(f"Found begin location field in PCI table: {begin_loc_field_pci}")
if end_loc_field_pci:
    print(f"Found end location field in PCI table: {end_loc_field_pci}")

# Check if we have the necessary fields
if not pci_street_id_field or not pci_section_id_field:
    print("ERROR: Could not find Street ID and Section ID fields in the PCI table.")
    # Use a raise statement here if you want to stop execution

if not street_sec_field and (not street_id_field or not section_id_field):
    print("ERROR: Could not find StreetSec or Street ID/Section ID fields in the feature layer.")
    # Use a raise statement here if you want to stop execution

# Check if PCI fields were found in the table
if not prev_insp_pci_field or not last_insp_pci_field:
    print("ERROR: Could not find Prev_Insp_PCI or Last_Insp_PCI fields in the PCI table.")
    # Use a raise statement here if you want to stop execution
else:
    print(f"Found both PCI fields in the table. Will calculate difference: {prev_insp_pci_field} - {last_insp_pci_field}")


Looking for key fields in the PCI Differences table...
Found street ID field: Street_ID
Found section ID field: Section_ID
Found previous inspection PCI field: Prev_Insp_PCI
Found last inspection PCI field: Last_Insp_PCI
Found previous inspection date field: Prev_Insp_Date
Found M&R treatment field: M_R_Treatment_Name
Found last inspection date field: Last_Insp_Date
Found begin location field in PCI table: Begin_Location
Found end location field in PCI table: End_Location
Found both PCI fields in the table. Will calculate difference: Prev_Insp_PCI - Last_Insp_PCI


In [59]:
# Create a dictionary to store PCI data for quicker lookups
print("\nReading PCI Differences data...")
pci_data = {}

# Determine which fields to fetch from the PCI table
pci_fields_to_fetch = [pci_street_id_field, pci_section_id_field]

# Add required fields if they exist
if prev_insp_pci_field:
    pci_fields_to_fetch.append(prev_insp_pci_field)
if last_insp_pci_field:
    pci_fields_to_fetch.append(last_insp_pci_field)
if prev_insp_date_field:
    pci_fields_to_fetch.append(prev_insp_date_field)
if m_r_date_field:
    pci_fields_to_fetch.append(m_r_date_field)
if m_r_treatment_field:
    pci_fields_to_fetch.append(m_r_treatment_field)
if last_insp_date_field:
    pci_fields_to_fetch.append(last_insp_date_field)
# Add begin/end location fields from PCI table if they exist
if begin_loc_field_pci:
    pci_fields_to_fetch.append(begin_loc_field_pci)
if end_loc_field_pci:
    pci_fields_to_fetch.append(end_loc_field_pci)

with arcpy.da.SearchCursor(pci_table, pci_fields_to_fetch) as cursor:
    for row in cursor:
        street_id = str(row[0]).strip()
        section_id = str(row[1]).strip()
        
        # Create key in "Street_ID - Section_ID" format
        key = f"{street_id} - {section_id}"
        
        record_data = {
            'street_id': street_id,
            'section_id': section_id
        }
        
        # Get field index
        field_idx = 2
        
        # Get PCI values if fields are available
        if prev_insp_pci_field:
            record_data['prev_pci'] = row[field_idx]
            field_idx += 1
        if last_insp_pci_field:
            record_data['last_pci'] = row[field_idx]
            field_idx += 1
        
        # Get date and treatment fields
        if prev_insp_date_field:
            record_data['prev_insp_date'] = row[field_idx]
            field_idx += 1
        if m_r_date_field:
            record_data['m_r_date'] = row[field_idx]
            field_idx += 1
        if m_r_treatment_field:
            record_data['m_r_treatment'] = row[field_idx]
            field_idx += 1
        if last_insp_date_field:
            record_data['last_insp_date'] = row[field_idx]
            field_idx += 1
        
        # Get begin/end location fields from PCI table if they exist
        if begin_loc_field_pci:
            record_data['begin_loc_pci'] = row[field_idx]
            field_idx += 1
        if end_loc_field_pci:
            record_data['end_loc_pci'] = row[field_idx]
            field_idx += 1
        
        # Calculate PCI difference if both PCI fields are available
        if 'prev_pci' in record_data and 'last_pci' in record_data:
            prev_pci = record_data['prev_pci']
            last_pci = record_data['last_pci']
            
            # Check for valid numeric values before calculating
            if prev_pci is not None and last_pci is not None:
                try:
                    pci_diff_calc = float(prev_pci) - float(last_pci)
                    record_data['pci_diff_calc'] = pci_diff_calc
                    
                    # Check if the record meets threshold criteria
                    meets_threshold = False
                    
                    # For lower threshold, we include regardless of work history
                    if pci_diff_calc <= lower_threshold:
                        meets_threshold = True
                        print(f"Record {key} included: PCI difference {pci_diff_calc} <= {lower_threshold}")
                    
                    # For higher threshold, we may filter based on work history
                    elif pci_diff_calc >= higher_threshold:
                        if work_history_filter == 'yes':
                            # Check if there is NO work history (empty treatment field)
                            has_work_history = 'm_r_treatment' in record_data and record_data['m_r_treatment']
                            if not has_work_history:
                                meets_threshold = True
                                print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold} with NO work history")
                            else:
                                print(f"Record {key} excluded: PCI difference {pci_diff_calc} >= {higher_threshold} but HAS work history")
                        else:
                            # Include all records above higher threshold
                            meets_threshold = True
                            print(f"Record {key} included: PCI difference {pci_diff_calc} >= {higher_threshold}")
                    else:
                        print(f"Record {key} excluded: PCI difference {pci_diff_calc} not within thresholds")
                    
                    # Store record if it meets criteria
                    if meets_threshold:
                        pci_data[key] = record_data
                        
                except (ValueError, TypeError):
                    print(f"Warning: Could not calculate PCI difference for {key}. Values: {prev_pci}, {last_pci}")

print(f"Loaded {len(pci_data)} records from PCI Differences table that meet threshold criteria.")


Reading PCI Differences data...
Record 100 - 1076 excluded: PCI difference 14.0 not within thresholds
Record 100 - 3084 excluded: PCI difference -1.0 not within thresholds
Record 100 - 3229 excluded: PCI difference 8.0 not within thresholds
Record 100 - 3232 excluded: PCI difference -15.0 not within thresholds
Record 100 - 3234 excluded: PCI difference 19.0 not within thresholds
Record 100 - 7032 excluded: PCI difference 17.0 not within thresholds
Record 100 - 7448 excluded: PCI difference -6.0 not within thresholds
Record 1000 - 1274 excluded: PCI difference -4.0 not within thresholds
Record 1000A - 1000 excluded: PCI difference 32.0 not within thresholds
Record 1002 - 5466 excluded: PCI difference -14.0 not within thresholds
Record 1002 - 5467 excluded: PCI difference -2.0 not within thresholds
Record 1004 - 1438 excluded: PCI difference 15.0 not within thresholds
Record 1004 - 3424 excluded: PCI difference -1.0 not within thresholds
Record 1004 - 4057 excluded: PCI difference 8.0 n

Record 1079 - 5620 excluded: PCI difference -4.0 not within thresholds
Record 1080 - 1799 excluded: PCI difference -5.0 not within thresholds
Record 1082 - 4046 excluded: PCI difference -4.0 not within thresholds
Record 1084 - 2298 excluded: PCI difference -1.0 not within thresholds
Record 1086 - 2301 excluded: PCI difference -1.0 not within thresholds
Record 1088 - 1340 excluded: PCI difference -24.0 not within thresholds
Record 1090 - 2048 excluded: PCI difference -15.0 not within thresholds
Record 1092 - 4794 excluded: PCI difference 0.0 not within thresholds
Record 1092 - 7124 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 1631 excluded: PCI difference -1.0 not within thresholds
Record 1094 - 2606 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 3412 excluded: PCI difference -3.0 not within thresholds
Record 1096 - 5056 excluded: PCI difference -4.0 not within thresholds
Record 1096 - 5057 excluded: PCI difference -4.0 not within thresholds
Recor

Record 1166 - 6400 excluded: PCI difference -5.0 not within thresholds
Record 1168 - 1513 excluded: PCI difference -1.0 not within thresholds
Record 1168 - 6992 excluded: PCI difference -1.0 not within thresholds
Record 1170 - 1412 excluded: PCI difference 20.0 not within thresholds
Record 1170 - 1415 included: PCI difference 46.0 >= 40.0 with NO work history
Record 1170 - 4278 excluded: PCI difference 4.0 not within thresholds
Record 1170 - 4279 excluded: PCI difference 13.0 not within thresholds
Record 1170 - 4280 excluded: PCI difference -11.0 not within thresholds
Record 1170 - 4281 excluded: PCI difference -3.0 not within thresholds
Record 1172 - 4514 included: PCI difference -71.0 <= -40.0
Record 1174 - 7482 excluded: PCI difference -23.0 not within thresholds
Record 1176 - 3497 excluded: PCI difference -28.0 not within thresholds
Record 1176 - 3503 excluded: PCI difference -21.0 not within thresholds
Record 1177 - 6499 excluded: PCI difference -3.0 not within thresholds
Record 1

Record 1282 - 7344 excluded: PCI difference -2.0 not within thresholds
Record 1284 - 3986 excluded: PCI difference -5.0 not within thresholds
Record 1284 - 3987 excluded: PCI difference -6.0 not within thresholds
Record 1284 - 3988 excluded: PCI difference -4.0 not within thresholds
Record 1284 - 3989 excluded: PCI difference -5.0 not within thresholds
Record 1284 - 4796 excluded: PCI difference -4.0 not within thresholds
Record 1284 - 4799 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 4800 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 5572 excluded: PCI difference -5.0 not within thresholds
Record 1284 - 5573 excluded: PCI difference -3.0 not within thresholds
Record 1284 - 5574 excluded: PCI difference -5.0 not within thresholds
Record 1286 - 5575 excluded: PCI difference -4.0 not within thresholds
Record 1288 - 3961 excluded: PCI difference -3.0 not within thresholds
Record 1290 - 2958 excluded: PCI difference -2.0 not within thresholds
Record

Record 135 - 3004 excluded: PCI difference -9.0 not within thresholds
Record 135 - 30041 excluded: PCI difference -6.0 not within thresholds
Record 135 - 3384 excluded: PCI difference -7.0 not within thresholds
Record 135 - 3385 excluded: PCI difference 10.0 not within thresholds
Record 135 - 3386 excluded: PCI difference -8.0 not within thresholds
Record 135 - 3387 excluded: PCI difference -1.0 not within thresholds
Record 135 - 34431 excluded: PCI difference -10.0 not within thresholds
Record 135 - 4015 excluded: PCI difference -10.0 not within thresholds
Record 135 - 4037 excluded: PCI difference 1.0 not within thresholds
Record 135 - 4103 excluded: PCI difference 2.0 not within thresholds
Record 135 - 4104 excluded: PCI difference 1.0 not within thresholds
Record 135 - 4177 excluded: PCI difference -10.0 not within thresholds
Record 135 - 4244 excluded: PCI difference -17.0 not within thresholds
Record 135 - 5272 excluded: PCI difference 1.0 not within thresholds
Record 135 - 5301 

...

Record 1750 - 6156 excluded: PCI difference -5.0 not within thresholds
Record 1750 - 6157 excluded: PCI difference -4.0 not within thresholds
Record 1750 - 7230 excluded: PCI difference -4.0 not within thresholds
Record 1752 - 1330 excluded: PCI difference -9.0 not within thresholds
Record 1754 - 3513 included: PCI difference -57.0 <= -40.0
Record 1756 - 4734 excluded: PCI difference -2.0 not within thresholds
Record 1758 - 1353 excluded: PCI difference -4.0 not within thresholds
Record 1758 - 4877 excluded: PCI difference -3.0 not within thresholds
Record 1758 - 4878 excluded: PCI difference -4.0 not within thresholds
Record 1760 - 5585 excluded: PCI difference -1.0 not within thresholds
Record 1760 - 5617 excluded: PCI difference -1.0 not within thresholds
Record 1762 - 1675 excluded: PCI difference -5.0 not within thresholds
Record 1764 - 1232 excluded: PCI difference -3.0 not within thresholds
Record 1764 - 3565 excluded: PCI difference -2.0 not within thresholds
Record 1764 - 6053

Record 1856 - 4844 excluded: PCI difference -13.0 not within thresholds
Record 1856 - 7313 excluded: PCI difference -7.0 not within thresholds
Record 1858 - 1480 excluded: PCI difference -15.0 not within thresholds
Record 1860 - 3888 excluded: PCI difference -35.0 not within thresholds
Record 1860 - 3889 excluded: PCI difference 5.0 not within thresholds
Record 1860 - 5146 excluded: PCI difference -28.0 not within thresholds
Record 1862 - 1494 excluded: PCI difference 8.0 not within thresholds
Record 1864 - 1241 excluded: PCI difference 31.0 not within thresholds
Record 1864 - 5438 excluded: PCI difference 39.0 not within thresholds
Record 1864 - 5439 excluded: PCI difference 28.0 not within thresholds
Record 1864 - 5440 excluded: PCI difference 46.0 >= 40.0 but HAS work history
Record 1864 - 5441 excluded: PCI difference -12.0 not within thresholds
Record 1864 - 7402 excluded: PCI difference 38.0 not within thresholds
Record 1866 - 1240 excluded: PCI difference 16.0 not within thresho

Record 1972 - 6358 excluded: PCI difference -5.0 not within thresholds
Record 1972 - 7282 excluded: PCI difference -4.0 not within thresholds
Record 1974 - 2066 excluded: PCI difference -5.0 not within thresholds
Record 1974 - 2067 excluded: PCI difference -5.0 not within thresholds
Record 1974 - 5760 excluded: PCI difference -2.0 not within thresholds
Record 1974 - 5761 excluded: PCI difference -4.0 not within thresholds
Record 1974 - 5762 excluded: PCI difference -4.0 not within thresholds
Record 1976 - 4818 excluded: PCI difference -5.0 not within thresholds
Record 1976 - 7535 excluded: PCI difference 2.0 not within thresholds
Record 1978 - 7261 excluded: PCI difference -4.0 not within thresholds
Record 1980 - 6959 excluded: PCI difference -12.0 not within thresholds
Record 1982 - 1490 excluded: PCI difference -2.0 not within thresholds
Record 1982 - 1491 excluded: PCI difference -3.0 not within thresholds
Record 1982 - 1492 excluded: PCI difference -1.0 not within thresholds
Record

Record 2048 - 6657 excluded: PCI difference -1.0 not within thresholds
Record 2048 - 6836 excluded: PCI difference -4.0 not within thresholds
Record 2048 - 6837 excluded: PCI difference -4.0 not within thresholds
Record 2048 - 7087 excluded: PCI difference -3.0 not within thresholds
Record 2048 - 7105 excluded: PCI difference -3.0 not within thresholds
Record 2048 - 7468 excluded: PCI difference -3.0 not within thresholds
Record 2048 - 7544 excluded: PCI difference -1.0 not within thresholds
Record 205 - 1692 excluded: PCI difference 1.0 not within thresholds
Record 2050 - 1707 excluded: PCI difference -5.0 not within thresholds
Record 2050 - 6873 excluded: PCI difference -6.0 not within thresholds
Record 2052 - 1928 excluded: PCI difference -13.0 not within thresholds
Record 2054 - 6923 excluded: PCI difference -12.0 not within thresholds
Record 2056 - 3722 excluded: PCI difference -1.0 not within thresholds
Record 2056 - 3723 excluded: PCI difference -1.0 not within thresholds
Record

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [60]:
# Outputs in default GDB
default_gdb = aprx.defaultGeodatabase
print(f"\nCreating outputs in default geodatabase: {default_gdb}")

# Create more user-friendly names
current_date = datetime.datetime.now().strftime("%Y%m%d")
formatted_thresholds = f"PCI_Diff_LE_{lower_threshold}_GE_{higher_threshold}"

# More recognizable names for outputs
table_name = f"QC_Table"
fc_name = f"QC_Points"
joined_fc_name = f"QC_Joined_Shapefile"

table_path = os.path.join(default_gdb, table_name)
fc_path = os.path.join(default_gdb, fc_name)
joined_fc_path = os.path.join(default_gdb, joined_fc_name)

# Thorough cleanup of ALL existing outputs with similar names to avoid duplicates
print("\nPerforming thorough cleanup of existing layers and tables...")

# Start by removing any matching layers from ALL maps in the project
for m in aprx.listMaps():
    print(f"Checking map: {m.name}")
    
    # Get lists of layers and tables to remove (collect first, remove after)
    layers_to_remove = []
    tables_to_remove = []
    
    # Check all layers
    for lyr in m.listLayers():
        try:
            if any(pattern in lyr.name for pattern in ["PCI_Threshold_Points", "PCI_Joined", "Midpoints", "TEMP_Midpoints"]):
                layers_to_remove.append(lyr)
                print(f"Will remove layer: {lyr.name}")
        except AttributeError:
            # Handle layers without valid name attribute
            layers_to_remove.append(lyr)
            print("Found a layer with no valid name attribute - will try to remove")
    
    # Check all tables
    for tbl in m.listTables():
        try:
            if "PCI_Threshold_Table" in tbl.name:
                tables_to_remove.append(tbl)
                print(f"Will remove table: {tbl.name}")
        except AttributeError:
            # Handle tables without valid name attribute
            tables_to_remove.append(tbl)
            print("Found a table with no valid name attribute - will try to remove")
    
    # Remove all identified layers
    for lyr in layers_to_remove:
        try:
            m.removeLayer(lyr)
            print(f"Removed layer from map")
        except Exception as e:
            print(f"Warning: Could not remove layer. Error: {str(e)}")
    
    # Remove all identified tables
    for tbl in tables_to_remove:
        try:
            m.removeTable(tbl)
            print(f"Removed table from map")
        except Exception as e:
            print(f"Warning: Could not remove table. Error: {str(e)}")

# Delete ALL matching feature classes and tables in the geodatabase
arcpy_workspace = aprx.defaultGeodatabase
arcpy.env.workspace = arcpy_workspace

# Define patterns to match ALL output types
pattern_list = [
    "*PCI_Threshold_Points*", 
    "*PCI_Threshold_Table*", 
    "*Midpoints*", 
    "*TEMP_Midpoints*",
    "*PCI_Joined*"
]

# Delete all items matching any pattern
for pattern in pattern_list:
    # Try to delete as feature classes
    for item in arcpy.ListFeatureClasses(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing feature class: {item}")
        except Exception as e:
            print(f"Warning: Could not delete feature class {item}. Error: {str(e)}")
    
    # Try to delete as tables
    for item in arcpy.ListTables(pattern):
        try:
            item_path = os.path.join(arcpy_workspace, item)
            arcpy.management.Delete(item_path)
            print(f"Deleted existing table: {item}")
        except Exception as e:
            print(f"Warning: Could not delete table {item}. Error: {str(e)}")

print("Cleanup complete - all previous outputs removed")

In [61]:
# Create a unique name for the temporary midpoints
unique_id = str(uuid.uuid4())[:8]
midpoints_fc_name = f"TEMP_Midpoints_{unique_id}"
midpoints_fc_path = os.path.join(default_gdb, midpoints_fc_name)

# Create midpoints using the FeatureToPoint tool
print("\nGenerating midpoints using FeatureToPoint tool...")
if arcpy.Exists(midpoints_fc_path):
    arcpy.management.Delete(midpoints_fc_path)
    
arcpy.management.FeatureToPoint(
    PMP_Path, 
    midpoints_fc_path, 
    "CENTROID"  # Using CENTROID for lines will give the midpoint
)

print(f"Midpoints created successfully at: {midpoints_fc_path}")


Generating midpoints using FeatureToPoint tool...
Midpoints created successfully at: C:\Users\JLin\Downloads\QC Tool\Default.gdb\TEMP_Midpoints_575349ca


In [62]:
# Determine fields to retrieve from the original feature class
shapefile_retrieve_fields = ["SHAPE@"]

# If we have a combined field, use it
if street_sec_field:
    shapefile_retrieve_fields.append(street_sec_field)
# Otherwise use separate fields
else:
    shapefile_retrieve_fields.append(street_id_field)
    shapefile_retrieve_fields.append(section_id_field)
    
# Add additional fields if they exist
if street_name_field:
    shapefile_retrieve_fields.append(street_name_field)
if begin_loc_field:
    shapefile_retrieve_fields.append(begin_loc_field)
if end_loc_field:
    shapefile_retrieve_fields.append(end_loc_field)

# Join fields from original data with midpoint geometries
# First, create a dictionary of attributes from the original feature class
attributes_dict = {}

with arcpy.da.SearchCursor(wildomar_layer, shapefile_retrieve_fields) as cursor:
    for i, row in enumerate(cursor):
        # Get the shape
        shape = row[0]  # SHAPE@
        
        # Initialize with defaults
        street_id_value = ""
        section_id_value = ""
        street_name_value = ""
        begin_loc_value = ""
        end_loc_value = ""
        
        field_idx = 1  # Start from field after SHAPE@
        
        # Determine the key based on available fields
        if street_sec_field:
            street_sec_value = str(row[field_idx]).strip()
            key = street_sec_value
            street_id_value = street_sec_value
            section_id_value = street_sec_value
            field_idx += 1
        else:
            street_id_value = str(row[field_idx]).strip()
            section_id_value = str(row[field_idx + 1]).strip()
            key = f"{street_id_value} - {section_id_value}"
            field_idx += 2
        
        # Get additional fields if they exist
        if street_name_field:
            street_name_value = row[field_idx]
            field_idx += 1
        if begin_loc_field:
            begin_loc_value = row[field_idx]
            field_idx += 1
        if end_loc_field:
            end_loc_value = row[field_idx]
            field_idx += 1
        
        # Calculate midpoint
        midpoint = shape.positionAlongLine(0.5, True)
        
        # Project the point to WGS84 for API URLs
        proj = midpoint.projectAs(wgs84)
        lon = proj.firstPoint.X
        lat = proj.firstPoint.Y
        
        # Store original coordinates
        original_x = midpoint.firstPoint.X
        original_y = midpoint.firstPoint.Y
        
        # Store attributes
        attributes_dict[i] = {
            'street_id': street_id_value,
            'section_id': section_id_value,
            'combined_key': key,
            'street_name': street_name_value,
            'begin_loc': begin_loc_value,
            'end_loc': end_loc_value,
            'latitude': lat,
            'longitude': lon,
            'original_x': original_x,
            'original_y': original_y,
            'spatial_reference': layer_sr
        }

In [63]:
# ================================================================
# Cell 11 – Build output records with QC_Points field names
# ================================================================

print("\n🔎 Matching attributes with PCI data (all features)…")

results = []

def safe_float(v):
    try:
        return float(v)
    except (TypeError, ValueError):
        return None

for fid, attrs in attributes_dict.items():
    key = attrs["combined_key"]
    rec = {}                               # fresh dict per line feature
    rec.update(attrs)                      # copy original line attributes

    # ---- Pull PCI‑history values -----------------------------------------
    pci = pci_data.get(key)                # may be None
    prev_date   = pci.get(prev_insp_date_field)   if pci else None
    prev_pci    = safe_float(pci.get(prev_insp_pci_field))  if pci else None
    mr_date     = pci.get(m_r_date_field)           if pci else None
    mr_treat    = pci.get(m_r_treatment_field)      if pci else None
    last_date   = pci.get(last_insp_date_field)     if pci else None
    last_pci    = safe_float(pci.get(last_insp_pci_field))  if pci else None

    rec["PrevInspDate"]    = prev_date
    rec["PrevInspPCI"]     = prev_pci
    rec["MRDate"]          = mr_date
    rec["MRTreatmentName"] = mr_treat
    rec["LastInspDate"]    = last_date
    rec["LastInspPCI"]     = last_pci

    # ---- Calculate diff & determine QC flag ------------------------------
    pci_diff = (last_pci - prev_pci) if (prev_pci is not None and last_pci is not None) else None
    rec["PCIDifference"] = pci_diff

    meets = False
    if pci_diff is not None:
        # case 1: NEGATIVE side
        if pci_diff <= lower_threshold:
            meets = True
        # case 2: POSITIVE side (check optional work‑history filter)
        elif pci_diff >= higher_threshold:
            if work_history_filter == "yes":
                # only keep if no MR treatment
                meets = (mr_treat is None or str(mr_treat).strip() == "")
            else:
                meets = True

    rec["QC"] = "Y" if meets else "N"
    results.append(rec)

print(f"✅ Created records for {len(results)} line features.")
print(f"   • {sum(r['QC']=='Y' for r in results)} meet the QC criteria.")



🔎 Matching attributes with PCI data (all features)…
✅ Created records for 6334 line features.
   • 0 meet the QC criteria.


In [64]:
# 1. Create the table with specified columns
print(f"Creating table: {table_name}")
arcpy.management.CreateTable(default_gdb, table_name)
arcpy.management.AddField(table_path, "StreetID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "SectionID", "TEXT", field_length=50)
arcpy.management.AddField(table_path, "StreetName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "BeginLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "EndLocation", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "PrevInspDate", "DATE")
arcpy.management.AddField(table_path, "PrevInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "MRDate", "DATE")
arcpy.management.AddField(table_path, "MRTreatmentName", "TEXT", field_length=100)
arcpy.management.AddField(table_path, "LastInspDate", "DATE")
arcpy.management.AddField(table_path, "LastInspPCI", "DOUBLE")
arcpy.management.AddField(table_path, "PCIDifference", "DOUBLE")  # Add PCI Difference field
arcpy.management.AddField(table_path, "Lat", "DOUBLE")
arcpy.management.AddField(table_path, "Long", "DOUBLE")
arcpy.management.AddField(table_path, "MapillaryLink", "TEXT", field_length=255)
arcpy.management.AddField(table_path, "GoogleImageLink", "TEXT", field_length=255)

# Insert the data into the table
print("Inserting data into the table...")
with arcpy.da.InsertCursor(table_path, [
        "StreetID", "SectionID", "StreetName", "BeginLocation", "EndLocation",
        "PrevInspDate", "PrevInspPCI", "MRDate", "MRTreatmentName", 
        "LastInspDate", "LastInspPCI", "PCIDifference", "Lat", "Long", 
        "MapillaryLink", "GoogleImageLink"
    ]) as ic:
    for r in results:
        lat = r['latitude']
        lon = r['longitude']
        
        # Simplified Mapillary link with no filters
        map_url = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        
        # Google Street View link
        g_url = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"
        
        # Get values with defaults for missing fields
        street_id = r.get('street_id', "")
        section_id = r.get('section_id', "")
        street_name = r.get('street_name', "")
        begin_loc = r.get('begin_loc', "")
        end_loc = r.get('end_loc', "")
        prev_insp_date = r.get('prev_insp_date', None)
        prev_insp_pci = r.get('prev_pci', None)
        m_r_date = r.get('m_r_date', None)
        m_r_treatment = r.get('m_r_treatment', "")
        last_insp_date = r.get('last_insp_date', None)
        last_insp_pci = r.get('last_pci', None)
        
        # Calculate PCI difference
        pci_difference = r.get('pci_diff_calc', None)
        
        # Debug output for begin and end locations
        print(f"Record {street_id}-{section_id}: BeginLoc={begin_loc}, EndLoc={end_loc}")
        
        ic.insertRow([
            street_id, section_id, street_name, begin_loc, end_loc,
            prev_insp_date, prev_insp_pci, m_r_date, m_r_treatment,
            last_insp_date, last_insp_pci, pci_difference, lat, lon,
            map_url, g_url
        ])

Creating table: QC_Table
Inserting data into the table...
Record 1002 - 5466-1002 - 5466: BeginLoc=, EndLoc=SW END
Record 1002 - 5467-1002 - 5467: BeginLoc=, EndLoc=NORTH END
Record 1004 - 4057-1004 - 4057: BeginLoc=, EndLoc=BEL AIR ST
Record 1004 - 5279-1004 - 5279: BeginLoc=, EndLoc=DALE AVE
Record 1004 - 1438-1004 - 1438: BeginLoc=, EndLoc=BENWOOD DR
Record 1004 - 3424-1004 - 3424: BeginLoc=, EndLoc=LAXORE ST
Record 1004 - 7494-1004 - 7494: BeginLoc=, EndLoc=BRODER ST
Record 1006 - 1287-1006 - 1287: BeginLoc=, EndLoc=ORANGEWOOD AVE
Record 1008 - 4045-1008 - 4045: BeginLoc=, EndLoc=EAST END
Record 1010 - 6775-1010 - 6775: BeginLoc=, EndLoc=NORTH END
Record 1012 - 5159-1012 - 5159: BeginLoc=, EndLoc=MANCHESTER AVE
Record 1014 - 4483-1014 - 4483: BeginLoc=, EndLoc=BROADWAY
Record 1014 - 5947-1014 - 5947: BeginLoc=, EndLoc=BROADWAY
Record 1016 - 6075-1016 - 6075: BeginLoc=, EndLoc=NE END
Record 1018 - 6573-1018 - 6573: BeginLoc=, EndLoc=REDROCK ST
Record 1018 - 6574-1018 - 6574: BeginLo

Record 1114 - 2053-1114 - 2053: BeginLoc=, EndLoc=EAST END
Record 1116 - 4575-1116 - 4575: BeginLoc=, EndLoc=FLIPPEN DR
Record 1118 - 1409-1118 - 1409: BeginLoc=, EndLoc=CHALET AVE
Record 1120 - 6161-1120 - 6161: BeginLoc=, EndLoc=RIVERVIEW DR
Record 1120 - 6162-1120 - 6162: BeginLoc=, EndLoc=BIRCHWOOD ST
Record 1122 - 1118-1122 - 1118: BeginLoc=, EndLoc=NORTH END
Record 1124 - 4725-1124 - 4725: BeginLoc=, EndLoc=OSHKOSH AVE
Record 1124 - 4726-1124 - 4726: BeginLoc=, EndLoc=LIZBETH AVE
Record 1126 - 5682-1126 - 5682: BeginLoc=, EndLoc=NORTH END
Record 1128 - 4823-1128 - 4823: BeginLoc=, EndLoc=NORTH END
Record 1130 - 1617-1130 - 1617: BeginLoc=, EndLoc=NORTH END
Record 1132 - 7083-1132 - 7083: BeginLoc=, EndLoc=FREEDOM AVE
Record 1134 - 6325-1134 - 6325: BeginLoc=, EndLoc=DUNE ST
Record 1134 - 2857-1134 - 2857: BeginLoc=, EndLoc=RIO VISTA ST
Record 1134 - 2858-1134 - 2858: BeginLoc=, EndLoc=MARJAN ST
Record 1136 - 1936-1136 - 1936: BeginLoc=, EndLoc=NORTH END
Record 1136 - 1937-1136 - 

Record 1254 - 5667-1254 - 5667: BeginLoc=, EndLoc=AVENIDA FELIPE @WYE
Record 1254 - 5668-1254 - 5668: BeginLoc=, EndLoc=AVENIDA MARGARITA
Record 1254 - 6664-1254 - 6664: BeginLoc=, EndLoc=AVENIDA FELIPE @WYE
Record 1256 - 5595-1256 - 5595: BeginLoc=, EndLoc=AVENIDA BERNARDO
Record 1258 - 6921-1258 - 6921: BeginLoc=, EndLoc=AVENIDA MALAGA
Record 1260 - 5765-1260 - 5765: BeginLoc=, EndLoc=RIO GRANDE DR
Record 1262 - 5758-1262 - 5758: BeginLoc=, EndLoc=AVENIDA JUAREZ
Record 1264 - 1345-1264 - 1345: BeginLoc=, EndLoc=CALLE DURANGO
Record 1266 - 5634-1266 - 5634: BeginLoc=, EndLoc=SMOKERIDGE TER
Record 1266 - 5635-1266 - 5635: BeginLoc=, EndLoc=AVENIDA FELIPE
Record 1266 - 7304-1266 - 7304: BeginLoc=, EndLoc=SMOKERIDGE TER
Record 1266 - 7305-1266 - 7305: BeginLoc=, EndLoc=AVENIDA FELIPE
Record 1268 - 6800-1268 - 6800: BeginLoc=, EndLoc=IMPERIAL HWY
Record 1270 - 2094-1270 - 2094: BeginLoc=, EndLoc=AVENIDA ARBOL
Record 1270 - 7339-1270 - 7339: BeginLoc=, EndLoc=CAMINO MANZANO
Record 1270 - 7

Record 1378 - 4148-1378 - 4148: BeginLoc=, EndLoc=MAGNOLIA AVE
Record 1380 - 4421-1380 - 4421: BeginLoc=, EndLoc=MULLER ST
Record 1382 - 4091-1382 - 4091: BeginLoc=, EndLoc=RIDGEWAY ST
Record 1384 - 4238-1384 - 4238: BeginLoc=, EndLoc=NORTH END
Record 1386 - 1391-1386 - 1391: BeginLoc=, EndLoc=WALNUT ST
Record 1386 - 1443-1386 - 1443: BeginLoc=, EndLoc=ASPEN ST
Record 1386 - 1444-1386 - 1444: BeginLoc=, EndLoc=PEPPER ST
Record 1388 - 1392-1388 - 1392: BeginLoc=, EndLoc=GILBUCK DR
Record 1390 - 1254-1390 - 1254: BeginLoc=, EndLoc=ARDEN ST
Record 1390 - 1094-1390 - 1094: BeginLoc=, EndLoc=FANN ST
Record 1390 - 4338-1390 - 4338: BeginLoc=, EndLoc=ECHO PL
Record 1390 - 4339-1390 - 4339: BeginLoc=, EndLoc=BROADVIEW ST
Record 1390 - 4341-1390 - 4341: BeginLoc=, EndLoc=NEPTUNE PL
Record 1390 - 4430-1390 - 4430: BeginLoc=, EndLoc=TRIDENT ST
Record 1390 - 5061-1390 - 5061: BeginLoc=, EndLoc=FALCON ST
Record 1390 - 5950-1390 - 5950: BeginLoc=, EndLoc=EAST END
Record 1390 - 5429-1390 - 5429: Begi

Record 1522 - 5636-1522 - 5636: BeginLoc=, EndLoc=BAUER RD
Record 1522 - 5637-1522 - 5637: BeginLoc=, EndLoc=CARNATION WAY
Record 1524 - 6240-1524 - 6240: BeginLoc=, EndLoc=NE END
Record 1526 - 2907-1526 - 2907: BeginLoc=, EndLoc=CHEVY CHASE DR
Record 1526 - 2908-1526 - 2908: BeginLoc=, EndLoc=ARLINGTON AVE
Record 1526 - 2909-1526 - 2909: BeginLoc=, EndLoc=BREWSTER AVE
Record 1526 - 2910-1526 - 2910: BeginLoc=, EndLoc=MALBORO AVE
Record 1528 - 2521-1528 - 2521: BeginLoc=, EndLoc=BLUEWATER CIR
Record 1528 - 2522-1528 - 2522: BeginLoc=, EndLoc=ALDERDALE AVE
Record 1530 - 1662-1530 - 1662: BeginLoc=, EndLoc=BAINBRIDGE AVE
Record 1530 - 1766-1530 - 1766: BeginLoc=, EndLoc=HOLTWOOD AVE
Record 1530 - 6745-1530 - 6745: BeginLoc=, EndLoc=ELKSTONE AVE
Record 1532 - 6808-1532 - 6808: BeginLoc=, EndLoc=NE END
Record 1532 - 2554-1532 - 2554: BeginLoc=, EndLoc=BLUEROCK ST
Record 1534 - 2683-1534 - 2683: BeginLoc=, EndLoc=KENWOOD AVE
Record 1534 - 6852-1534 - 6852: BeginLoc=, EndLoc=FOREST LN
Record

...

Record 2002 - 1363-2002 - 1363: BeginLoc=, EndLoc=WATER ST
Record 2002 - 1364-2002 - 1364: BeginLoc=, EndLoc=VALENCIA AVE
Record 2002 - 1183-2002 - 1183: BeginLoc=, EndLoc=SOUTH ST
Record 2002 - 1276-2002 - 1276: BeginLoc=, EndLoc=SANTA ANA ST
Record 2002 - 3566-2002 - 3566: BeginLoc=, EndLoc=LORRAINE DR
Record 2002 - 3567-2002 - 3567: BeginLoc=, EndLoc=FLORENCE AVE
Record 2002 - 3568-2002 - 3568: BeginLoc=, EndLoc=NARDA ST
Record 2002 - 3569-2002 - 3569: BeginLoc=, EndLoc=VERMONT AVE
Record 2004 - 1924-2004 - 1924: BeginLoc=, EndLoc=CLIFTON AVE
Record 2006 - 1137-2006 - 1137: BeginLoc=, EndLoc=BALL RD
Record 2008 - 1512-2008 - 1512: BeginLoc=, EndLoc=LA PALMA AVE
Record 5720 - 7551-5720 - 7551: BeginLoc=, EndLoc=NORTH END
Record 2010 - 5689-2010 - 5689: BeginLoc=, EndLoc=OSHKOSH AVE
Record 2012 - 6580-2012 - 6580: BeginLoc=, EndLoc=NE END
Record 2014 - 1237-2014 - 1237: BeginLoc=, EndLoc=NUTWOOD ST
Record 2016 - 5402-2016 - 5402: BeginLoc=, EndLoc=PRIMROSE ST
Record 2018 - 3639-2018 -

Record 2122 - 7438-2122 - 7438: BeginLoc=, EndLoc=GRAND AVE
Record 2124 - 4005-2124 - 4005: BeginLoc=, EndLoc=MOONSTONE ST
Record 2126 - 4147-2126 - 4147: BeginLoc=, EndLoc=EAST END
Record 2128 - 4144-2128 - 4144: BeginLoc=, EndLoc=COLGATE ST
Record 2130 - 2447-2130 - 2447: BeginLoc=, EndLoc=TRANSIT AVE
Record 2132 - 3475-2132 - 3475: BeginLoc=, EndLoc=BROADWAY
Record 2134 - 2155-2134 - 2155: BeginLoc=, EndLoc=NORTH END
Record 2136 - 6765-2136 - 6765: BeginLoc=, EndLoc=NORTH END
Record 2138 - 6471-2138 - 6471: BeginLoc=, EndLoc=NORTH END
Record 2140 - 6314-2140 - 6314: BeginLoc=, EndLoc=RED GUM ST
Record 2140 - 6698-2140 - 6698: BeginLoc=, EndLoc=WEST END
Record 2140 - 7150-2140 - 7150: BeginLoc=, EndLoc=RED GUM ST
Record 2140 - 2230-2140 - 2230: BeginLoc=, EndLoc=SIMON CIR
Record 2140 - 1624-2140 - 1624: BeginLoc=, EndLoc=KRAEMER BLVD
Record 2140 - 1223-2140 - 1223: BeginLoc=, EndLoc=VAN HORNE WAY
Record 2140 - 2528-2140 - 2528: BeginLoc=, EndLoc=KRAEMER BLVD
Record 2140 - 2529-2140 -

Record 5398 - 1700-5398 - 1700: BeginLoc=, EndLoc=BURTON PL
Record 5398 - 1947-5398 - 1947: BeginLoc=, EndLoc=VIA BURTON ST @WYE
Record 5398 - 1592-5398 - 1592: BeginLoc=, EndLoc=STATE COLLEGE BLVD
Record 5398 - 1596-5398 - 1596: BeginLoc=, EndLoc=PLACENTIA AVE
Record 5400 - 2033-5400 - 2033: BeginLoc=, EndLoc=SOUTH END
Record 5400 - 5770-5400 - 5770: BeginLoc=, EndLoc=PASEO CABALLO
Record 5400 - 7191-5400 - 7191: BeginLoc=, EndLoc=VIA CORRAL @WYE
Record 5400 - 7199-5400 - 7199: BeginLoc=, EndLoc=CIRCULO LAZO
Record 5402 - 7259-5402 - 7259: BeginLoc=, EndLoc=NW END
Record 5402 - 5731-5402 - 5731: BeginLoc=, EndLoc=SANTA ANA CANYON RD
Record 5404 - 7225-5404 - 7225: BeginLoc=, EndLoc=AVENIDA DE SANTIAGO
Record 5406 - 7192-5406 - 7192: BeginLoc=, EndLoc=PASEORENA
Record 5406 - 6355-5406 - 6355: BeginLoc=, EndLoc=PASEO REAL
Record 5406 - 2057-5406 - 2057: BeginLoc=, EndLoc=PASEO REAL
Record 5407 - 6356-5407 - 6356: BeginLoc=, EndLoc=PASEO CELESTE
Record 5407 - 6357-5407 - 6357: BeginLoc=,

Record 5492 - 3530-5492 - 3530: BeginLoc=, EndLoc=OHIO ST
Record 5492 - 3531-5492 - 3531: BeginLoc=, EndLoc=RESH ST
Record 5492 - 3532-5492 - 3532: BeginLoc=, EndLoc=JANSS ST
Record 5492 - 3533-5492 - 3533: BeginLoc=, EndLoc=PINE ST
Record 5492 - 3534-5492 - 3534: BeginLoc=, EndLoc=HARBOR BLVD
Record 5492 - 2716-5492 - 2716: BeginLoc=, EndLoc=CLAUDINA ST
Record 5492 - 7591-5492 - 7591: BeginLoc=, EndLoc=MELROSE ST
Record 5492 - 7592-5492 - 7592: BeginLoc=, EndLoc=PHILADELPHIA ST
Record 7576 - 5490-7576 - 5490: BeginLoc=, EndLoc=KROEGER ST (N)
Record 75767 - 5492-75767 - 5492: BeginLoc=, EndLoc=KROEGER ST (S)
Record 5492 - 7577-5492 - 7577: BeginLoc=, EndLoc=KROEGER ST (S)
Record 5494 - 7479-5494 - 7479: BeginLoc=, EndLoc=WAKEFIELD AVE
Record 5494 - 4563-5494 - 4563: BeginLoc=, EndLoc=TONIA CT
Record 5494 - 4564-5494 - 4564: BeginLoc=, EndLoc=FLIPPEN CT
Record 5494 - 3945-5494 - 3945: BeginLoc=, EndLoc=ORANGEWOOD AVE
Record 5496 - 6048-5496 - 6048: BeginLoc=, EndLoc=CITY BOUNDARY
Record

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [65]:
# ================================================================
# Cell 13 – Create & populate QC_Points feature class
# ================================================================

print(f"\nCreating points feature class in WGS84: {fc_name}")
arcpy.management.CreateFeatureclass(
    default_gdb, fc_name, "POINT", spatial_reference=wgs84
)

# ---- Add fields -------------------------------------------------
field_defs = [
    ("StreetID",          "TEXT",   50),
    ("SectionID",         "TEXT",   50),
    ("StreetName",        "TEXT",  100),
    ("BeginLocation",     "TEXT",  100),
    ("EndLocation",       "TEXT",  100),
    ("PrevInspDate",      "DATE"),
    ("PrevInspPCI",       "DOUBLE"),
    ("MRDate",            "DATE"),
    ("MRTreatmentName",   "TEXT",  100),
    ("LastInspDate",      "DATE"),
    ("LastInspPCI",       "DOUBLE"),
    ("PCIDifference",     "DOUBLE"),
    ("Lat",               "DOUBLE"),
    ("Long",              "DOUBLE"),
    ("MapillaryLink",     "TEXT", 255),
    ("GoogleImageLink",   "TEXT", 255),
    ("QC",                "TEXT",   1)   # <‑‑ NEW
]
for name, ftype, *rest in field_defs:
    length = rest[0] if rest else None
    arcpy.management.AddField(fc_path, name, ftype, field_length=length)

# ---- Insert rows ------------------------------------------------
print("Populating the points feature class…")
with arcpy.da.InsertCursor(
        fc_path,
        ["SHAPE@", "StreetID", "SectionID", "StreetName", "BeginLocation",
         "EndLocation", "PrevInspDate", "PrevInspPCI", "MRDate",
         "MRTreatmentName", "LastInspDate", "LastInspPCI",
         "PCIDifference", "Lat", "Long", "MapillaryLink",
         "GoogleImageLink", "QC"]        # <‑‑ QC added
) as curs:
    for r in results:
        pt   = arcpy.Point(r["longitude"], r["latitude"])
        geom = arcpy.PointGeometry(pt, wgs84)

        lat, lon  = r["latitude"], r["longitude"]
        map_link  = f"https://www.mapillary.com/app/?lat={lat}&lng={lon}&z=18"
        gsv_link  = f"https://www.google.com/maps/@?api=1&map_action=pano&viewpoint={lat},{lon}"

        curs.insertRow([
            geom,
            r.get("street_id"), r.get("section_id"), r.get("street_name"),
            r.get("begin_loc"), r.get("end_loc"),
            r.get("PrevInspDate"), r.get("PrevInspPCI"), r.get("MRDate"),
            r.get("MRTreatmentName"), r.get("LastInspDate"),
            r.get("LastInspPCI"), r.get("PCIDifference"),
            lat, lon, map_link, gsv_link, r["QC"]     # <‑‑ QC value
        ])

print("✅ Points created & attributes populated.")



Creating points feature class in WGS84: QC_Points
Populating the points feature class…
✅ Points created & attributes populated.


In [66]:
# Define the add_layer_and_style function first
def add_layer_and_style(layer_path, layer_type="POINT", color=None, size=None, use_tear_pin=False):
    """
    Add a layer to the map only if it doesn't already exist, and apply styling.
    Added support for Tear Pin symbology for points.
    Returns the layer if it was added, None otherwise.
    """
    print(f"Adding layer: {os.path.basename(layer_path)}")
    
    # Check if layer already exists in the map
    existing_layer = None
    for lyr in active_map.listLayers():
        try:
            if lyr.dataSource == layer_path:
                existing_layer = lyr
                print(f"Layer already exists in map: {lyr.name}")
                break
        except:
            continue
    
    # If layer doesn't exist, add it
    if not existing_layer:
        active_map.addDataFromPath(layer_path)
        
        # Find the newly added layer
        for lyr in active_map.listLayers():
            try:
                if lyr.dataSource == layer_path:
                    existing_layer = lyr
                    print(f"Added layer to map: {lyr.name}")
                    break
            except:
                continue
    
    # Style the layer if found
    if existing_layer:
        if layer_type == "POINT" and use_tear_pin:
            try:
                # Attempt to set the Tear Pin symbol
                print("Attempting to apply Tear Pin symbol...")
                
                # Method 1: Try using the symbology classes
                try:
                    symb = existing_layer.symbology
                    
                    # Step 1: Get the current symbol
                    if hasattr(symb, 'renderer') and hasattr(symb.renderer, 'symbol'):
                        # Try to change the symbol style
                        try:
                            # Change the symbol type to a marker symbol
                            symb.renderer.symbol.style = "Tear Pin 1"
                            
                            # Set the size if provided
                            if size:
                                symb.renderer.symbol.size = size
                            else:
                                symb.renderer.symbol.size = 12  # Default tear pin size
                                
                            # Set the color if provided
                            if color:
                                symb.renderer.symbol.color = {'RGB': color}
                                
                            # Apply the symbology
                            existing_layer.symbology = symb
                            print("Applied Tear Pin symbol to point layer")
                        except Exception as e:
                            print(f"Could not set Tear Pin style: {str(e)}")
                            
                            # Try alternative method - just set size and color
                            try:
                                if size:
                                    symb.renderer.symbol.size = size
                                if color:
                                    symb.renderer.symbol.color = {'RGB': color}
                                existing_layer.symbology = symb
                                print("Applied basic styling (size/color) to point layer")
                                print("Please manually change the symbol to Tear Pin 1:")
                                print("1. Right-click the layer → Symbology")
                                print("2. Click on the symbol")
                                print("3. In the Format Point Symbol pane, select 'Tear Pin 1' from the gallery")
                            except:
                                print("Could not apply even basic styling")
                except Exception as e:
                    print(f"Could not modify symbol: {str(e)}")
                    print("Please manually set the Tear Pin symbol:")
                    print("1. Right-click the point layer in the Contents pane")
                    print("2. Select 'Symbology'")
                    print("3. Click on the current symbol to open the symbol selector")
                    print("4. Choose 'Tear Pin 1' from the gallery")
                    print("5. Set the appropriate size and color")
            except Exception as e:
                print(f"Could not apply Tear Pin styling: {str(e)}")
        elif color:  # Regular styling for other layer types or when not using Tear Pin
            try:
                symb = existing_layer.symbology
                if hasattr(symb, 'renderer'):
                    if hasattr(symb.renderer, 'symbol'):
                        if color:
                            symb.renderer.symbol.color = {'RGB': color}
                        if size and layer_type == "POINT":
                            symb.renderer.symbol.size = size
                        existing_layer.symbology = symb
                        print(f"Applied styling to layer: {existing_layer.name}")
            except Exception as e:
                print(f"Could not apply styling: {str(e)}")
    
    return existing_layer

# Now use the function to add layers only if needed, with proper styling
print("\nAdding outputs to the map (avoiding duplicates)...")

# Add point features with red styling
point_layer = add_layer_and_style(fc_path, "POINT", [255, 0, 0, 100], 8)

# Add table (tables don't need styling)
print("Adding table to map...")
existing_table = None
for tbl in active_map.listTables():
    try:
        if tbl.dataSource == table_path:
            existing_table = tbl
            print(f"Table already exists in map: {tbl.name}")
            break
    except:
        continue

if not existing_table:
    active_map.addDataFromPath(table_path)
    print(f"Added table to map: {table_name}")

# Save the project to preserve changes
aprx.save()
print("Project saved successfully with all changes.")
print("\nScript completed successfully!")
print(f"Created table: {table_name}")
print(f"Created feature class: {fc_name}")
print(f"Found {len(results)} points that met the threshold criteria.")

# Summary report of Begin/End Location fields processing
print("\n===== Begin/End Location Fields Summary =====")
begin_loc_feature_found = "Yes" if begin_loc_field else "No"
end_loc_feature_found = "Yes" if end_loc_field else "No"
begin_loc_pci_found = "Yes" if begin_loc_field_pci else "No"
end_loc_pci_found = "Yes" if end_loc_field_pci else "No"

print(f"Begin Location field found in feature layer: {begin_loc_feature_found}")
print(f"End Location field found in feature layer: {end_loc_feature_found}")
print(f"Begin Location field found in PCI table: {begin_loc_pci_found}")
print(f"End Location field found in PCI table: {end_loc_pci_found}")
print("============================================")


Adding outputs to the map (avoiding duplicates)...
Adding layer: QC_Points
Layer already exists in map: QC_Points
Applied styling to layer: QC_Points
Adding table to map...
Table already exists in map: QC_Table
Project saved successfully with all changes.

Script completed successfully!
Created table: QC_Table
Created feature class: QC_Points
Found 6334 points that met the threshold criteria.

===== Begin/End Location Fields Summary =====
Begin Location field found in feature layer: No
End Location field found in feature layer: Yes
Begin Location field found in PCI table: Yes
End Location field found in PCI table: Yes


In [67]:
# This cell is optional - you can create a wrapper function to call everything
def find_midpoints_with_pci_matching():
    """Main function to execute the workflow"""
    try:
        # All the code from cells 3-15 would go here
        print("Completed successfully!")
    except Exception as e:
        import traceback
        print(f"ERROR: An exception occurred during script execution:\n{str(e)}")
        print(traceback.format_exc())
        
        # Try to save any incomplete but important data
        try:
            if 'aprx' in locals():
                aprx.save()
                print("Project saved despite error.")
        except:
            print("Could not save project after error.")

# Uncomment the line below if you want to run everything at once
# find_midpoints_with_pci_matching()

In [68]:
# ---------------------------------------------------------------
# Spatial‑join back to the original line feature
# ---------------------------------------------------------------
print("\n===== Performing Spatial Join to Original Line Feature =====")

# 1) Field mapping ------------------------------------------------
print("Setting up field mapping...")
field_mapping = arcpy.FieldMappings()
field_mapping.addTable(PMP_Path)           # schema of the line feature

selected_fields = [                       # carry these across from points
    {"name": "PrevInspDate",    "alias": "Previous_Date"},
    {"name": "PrevInspPCI",     "alias": "Previous_PCI"},
    {"name": "MRDate",          "alias": "MR_Date"},
    {"name": "MRTreatmentName", "alias": "MR_Treatment"},
    {"name": "LastInspDate",    "alias": "Last_Insp_Date"},
    {"name": "LastInspPCI",     "alias": "Last_Insp_PCI"},
    {"name": "PCIDifference",   "alias": "PCI_Diff"},
    {"name": "MapillaryLink",   "alias": "Mapillary"},
    {"name": "GoogleImageLink", "alias": "Google"},
    {"name": "QC",              "alias": "QC"}          # <‑‑ NEW
]

for fld in selected_fields:
    fm = arcpy.FieldMap()
    fm.addInputField(fc_path, fld["name"])
    out_fld          = fm.outputField
    out_fld.name     = fld["alias"]
    out_fld.aliasName = fld["alias"]
    fm.outputField   = out_fld
    field_mapping.addFieldMap(fm)

# 2) Make sure the joined FC will have a QC field -----------------
qc_field = "QC"
if not arcpy.ListFields(PMP_Path, qc_field):
    arcpy.management.AddField(PMP_Path, qc_field, "TEXT", field_length=1)

# 3) Run the spatial join ----------------------------------------
print(f"Creating joined feature class: {joined_fc_name}")
arcpy.analysis.SpatialJoin(
    target_features=PMP_Path,
    join_features=fc_path,
    out_feature_class=joined_fc_path,
    join_operation="JOIN_ONE_TO_ONE",
    join_type="KEEP_ALL",
    field_mapping=field_mapping,
    match_option="CLOSEST",
    search_radius="10 Meters"
)

# 4) (No post‑join QC recalculation needed – values already there)

print("✅ Spatial join complete – QC values carried through.")



===== Performing Spatial Join to Original Line Feature =====
Setting up field mapping...
Creating joined feature class: QC_Joined_Shapefile
✅ Spatial join complete – QC values carried through.
